# Week 4

# Import dependencies

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
#Q9
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score


* In case of the following error while calling `pd.read_excel()` function:
```
ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
```
    * Do as it says: `pip install openpyxl`

In [50]:
#!pip install openpyxl

# Setup

### Classification task
* classifying tropical fruit types from 34 features extracted from the images of fruits.
* Types present: {Barhee, Deglet Nour, Sukkary, Rotab Mozafati, Ruthana, Safawi, and Sagai}
* Use-case: Given the 34 features from a new image of a fruit and pass it to the classifier, the classifier will be able to tell its type.

In [2]:
df = pd.read_excel('datasets/classification-task/F.xlsx', sheet_name='Sheet_02')


* This method is particularly useful during the initial stages of data analysis, 
* It gives you a quick overview of the dataset's structure 
* Makes informed decisions about data cleaning such as 
    1. transformation 
    2. further analysis steps.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           898 non-null    int64  
 1   PERIMETER      898 non-null    float64
 2   MAJOR_AXIS     898 non-null    float64
 3   MINOR_AXIS     898 non-null    float64
 4   ECCENTRICITY   898 non-null    float64
 5   EQDIASQ        898 non-null    float64
 6   SOLIDITY       898 non-null    float64
 7   CONVEX_AREA    898 non-null    int64  
 8   EXTENT         898 non-null    float64
 9   ASPECT_RATIO   898 non-null    float64
 10  ROUNDNESS      898 non-null    float64
 11  COMPACTNESS    898 non-null    float64
 12  SHAPEFACTOR_1  898 non-null    float64
 13  SHAPEFACTOR_2  898 non-null    float64
 14  SHAPEFACTOR_3  898 non-null    float64
 15  SHAPEFACTOR_4  898 non-null    float64
 16  MeanRR         898 non-null    float64
 17  MeanRG         898 non-null    float64
 18  MeanRB    

## df.describe()
It returns a DataFrame that includes the following statistics for each numerical column:

* Count: The number of non-null entries.
* Mean: The average value.
* Standard Deviation (std): The spread of the data.
* Minimum (min): The smallest value.
* 25th Percentile (25%): The value below which 25% of the data falls.
* 50th Percentile (50%): The median value.
* 75th Percentile (75%): The value below which 75% of the data falls.
* Maximum (max): The largest value.

This method is highly useful for getting a quick statistical summary of your dataset, which can help you understand the distribution and variability of your data. It can also be customized using parameters such as percentiles, include, and exclude to tailor the output to specific needs. For example, you can specify additional percentiles to be included in the summary or include/exclude certain data types.

* Overall, df.describe() is an essential tool for exploratory data analysis, providing a comprehensive overview of the dataset's key statistical properties.

In [4]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,...,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,...,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,...,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,...,-1.029100,1.708200,1.607600,1.767200,-1.091224e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,...,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,...,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,...,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,...,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


* The `df.head(n=5)` function is used to display the first `n` rows of a DataFrame `df`. 
* By default, `n` is set to 5, so it will display the first 5 rows of the DataFrame. 
* This function is commonly used to quickly inspect the structure and contents of a DataFrame. 
* It can be helpful to get a glimpse of the data and check if it is loaded correctly, if the columns are as expected, and if any data cleaning or preprocessing steps are needed. 
* The output will include the column names and the corresponding values for each row, providing a snapshot of the DataFrame's contents.

In [5]:
df.head(n=5)

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


# Q-1


* How many samples are there in the dataset?
* Also, how many features are there per sample, excluding the class/type label?
* Print the number of unique classes (i.e., fruit types), 
* List the class names
* Frequency distribution of the classes list previously.

## How many samples are there in the dataset?

In [6]:
num_samples = df.shape[0]  # Number of rows
print(f"Number of samples: {num_samples}")
num_columns = df.shape[1]  # Number of columns
print(f"Number of columns: {num_columns}")

Number of samples: 898
Number of columns: 35


The `df.shape` attribute in pandas DataFrame returns a tuple representing the dimensions of the DataFrame.
The first element of the tuple represents the number of rows (samples) in the DataFrame, while the second element represents the number of columns (features). 
By accessing `df.shape[0]`, we can obtain the number of samples in the DataFrame, and by accessing `df.shape[1]`, we can obtain the number of features (excluding the class/type label) in the DataFrame.
This attribute is useful for understanding the size and structure of the DataFrame, allowing us to perform various operations and analyses on the data.

In [7]:
df.shape

(898, 35)

## How many features are there per sample, excluding the class/type label?

In [8]:

num_features = df.shape[1] - 1  # Subtract 1 to exclude the target column
print(f"Number of features per sample (excluding target): {num_features}")

Number of features per sample (excluding target): 34


## Number of unique classes (fruit types) 


In [9]:
# Display the column names to identify the correct column for fruit types
#print(df.columns)

unique_classes = df['Class'].nunique()
print(f"Number of unique classes: {unique_classes}")

Number of unique classes: 7


In [10]:
df['Class'].nunique() #--> number of unique values

7

## List the class names

In [11]:
class_names = df['Class'].unique()
print(f"Class names: {class_names}")

Class names: ['BERHI' 'DEGLET' 'DOKOL' 'IRAQI' 'ROTANA' 'SAFAVI' 'SOGAY']


In [12]:
df['Class'].unique() ## --> list of unique values

array(['BERHI', 'DEGLET', 'DOKOL', 'IRAQI', 'ROTANA', 'SAFAVI', 'SOGAY'],
      dtype=object)

## Frequency distribution of each class

In [13]:
class_distribution = df['Class'].value_counts()
print(f"Frequency distribution of classes:\n{class_distribution}")

Frequency distribution of classes:
Class
DOKOL     204
SAFAVI    199
ROTANA    166
DEGLET     98
SOGAY      94
IRAQI      72
BERHI      65
Name: count, dtype: int64


In [14]:
df['Class'].value_counts() #--> class distribution


Class
DOKOL     204
SAFAVI    199
ROTANA    166
DEGLET     98
SOGAY      94
IRAQI      72
BERHI      65
Name: count, dtype: int64

# Q-2


* List mean, stdev, min, max of each of the features in the dataset

In [15]:
# Exclude the 'Class' column as it's the target
features = df.drop(columns=['Class'])

In [16]:
# Calculate the mean, standard deviation, min, and max for each feature
mean_values = features.mean()
std_values = features.std()
min_values = features.min()
max_values = features.max()

In [17]:
# Combine them into a DataFrame for better readability
summary_stats = pd.DataFrame({
    'Mean': mean_values,
    'Standard Deviation': std_values,
    'Min': min_values,
    'Max': max_values
})

In [18]:
# Display the summary statistics
print(summary_stats)

                       Mean  Standard Deviation           Min           Max
AREA           2.982952e+05        1.072452e+05  1.987000e+03  5.460630e+05
PERIMETER      2.057661e+03        4.100125e+02  9.118280e+02  2.811997e+03
MAJOR_AXIS     7.508120e+02        1.440593e+02  3.367227e+02  1.222723e+03
MINOR_AXIS     4.958728e+02        1.142689e+02  2.283200e+00  7.664536e+02
ECCENTRICITY   7.374678e-01        8.872739e-02  3.448000e-01  1.000000e+00
EQDIASQ        6.045779e+02        1.195939e+02  5.029840e+01  8.338279e+02
SOLIDITY       9.818403e-01        1.815742e-02  8.366000e-01  9.974000e-01
CONVEX_AREA    3.038456e+05        1.088157e+05  2.257000e+03  5.525980e+05
EXTENT         7.362671e-01        5.374518e-02  5.123000e-01  8.562000e-01
ASPECT_RATIO   2.131102e+00        1.782078e+01  1.065300e+00  5.355257e+02
ROUNDNESS      8.577200e-01        7.083882e-02  4.800000e-03  9.773000e-01
COMPACTNESS    8.071903e-01        6.217535e-02  4.110000e-02  9.681000e-01
SHAPEFACTOR_

In [19]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,...,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,...,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,...,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,...,-1.029100,1.708200,1.607600,1.767200,-1.091224e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,...,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,...,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,...,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,...,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


In [20]:
#save 
display_opt = {'display.max_rows': pd.get_option('display.max_rows'),
                'display.max_columns': pd.get_option('display.max_columns'),
                }

In [21]:
pd.set_option('display.max_columns',None)

In [22]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,MeanRR,MeanRG,MeanRB,StdDevRR,StdDevRG,StdDevRB,SkewRR,SkewRG,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,0.857720,0.807190,0.003428,0.001794,0.655420,0.988680,100.165885,97.609401,96.194889,29.047436,26.383362,26.148330,0.089266,0.564139,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,0.070839,0.062175,0.020456,0.000428,0.094314,0.020473,32.126549,28.251801,21.627409,6.306931,5.462164,4.905078,0.943285,1.039813,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,0.004800,0.041100,0.001700,0.001100,0.001700,0.756800,30.382600,41.048000,44.256900,7.303800,8.655700,5.148600,-1.724200,-1.834400,-1.029100,1.708200,1.607600,1.767200,-1.091224e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,0.827750,0.768050,0.002200,0.001500,0.589975,0.989300,76.448000,77.306125,78.502375,24.901525,22.289650,22.890975,-0.549900,-0.230200,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,0.867750,0.804950,0.002600,0.001600,0.647950,0.993600,107.682450,100.676000,99.225600,29.709450,25.638300,26.469000,-0.162700,0.243750,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,0.899500,0.848875,0.003200,0.002075,0.720625,0.996400,126.127450,119.159600,113.332250,33.274375,29.905325,29.482375,0.471025,1.406550,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,0.977300,0.968100,0.615400,0.004300,0.937300,0.999500,159.649400,166.135400,148.211400,48.571200,45.510700,42.422800,3.223600,3.697100,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


# Q-3


* Shuffle the data samples

In [23]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

* Display the first few rows of the shuffled dataset

In [24]:
print(df_shuffled.head())

     AREA  PERIMETER  MAJOR_AXIS  MINOR_AXIS  ECCENTRICITY   EQDIASQ  \
0  143957   1452.262    523.1368    352.3305        0.7392  428.1259   
1  260563   1975.374    762.5055    437.8510        0.8187  575.9854   
2  154245   1503.598    546.4366    361.8018        0.7494  443.1601   
3  317794   2280.969    875.5868    525.3380        0.8000  636.1037   
4  402279   2295.863    798.9365    643.2753        0.5931  715.6797   

   SOLIDITY  CONVEX_AREA  EXTENT  ASPECT_RATIO  ROUNDNESS  COMPACTNESS  \
0    0.9844       146236  0.7746        1.4848     0.8577       0.8184   
1    0.9871       263977  0.6648        1.7415     0.8391       0.7554   
2    0.9883       156070  0.7420        1.5103     0.8573       0.8110   
3    0.9016       352462  0.6484        1.6667     0.7676       0.7265   
4    0.9936       404887  0.7867        1.2420     0.9591       0.8958   

   SHAPEFACTOR_1  SHAPEFACTOR_2  SHAPEFACTOR_3  SHAPEFACTOR_4    MeanRR  \
0         0.0036         0.0024         0.6697 

# Q-4

* Split the dataset into training (80%) and test (20%)
* Separate the features and the target variable


In [25]:
X = df_shuffled.drop(columns=['Class'])  # Features
y = df_shuffled['Class']  # Target (fruit type)

* Split the dataset into 80% training and 20% testing

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* Check the sizes of the training and testing sets

In [27]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 718
Test set size: 180


# Q-5


* Scale all independent features.

### Explanation:
* fit_transform(X_train):Fits the scaler on the training data and transforms it.
* transform(X_test): Transforms the test data using the already fitted scaler (you should only fit on the training data to avoid data leakage).
* The scaled data is converted back into a pandas DataFrame for easier interpretation, although the raw numpy arrays work fine for most machine learning models.

This will ensure that all independent features are standardized, which helps many machine learning models perform better.

## Initialize the scaler

In [28]:
scaler = StandardScaler()

## Fit the scaler on the training data and transform both training and test sets

In [29]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Optionally, convert the scaled data back into DataFrame format

In [30]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

## Display the first few rows of the scaled training set

In [31]:
print(X_train_scaled.head())

       AREA  PERIMETER  MAJOR_AXIS  MINOR_AXIS  ECCENTRICITY   EQDIASQ  \
0 -0.521983  -0.564895   -0.702700   -0.182638     -0.603875 -0.410309   
1  0.720438   0.756000    0.523080    0.765270     -0.340188  0.726331   
2 -1.362303  -1.439936   -1.421520   -1.223381      0.267295 -1.372499   
3  1.037902   1.093054    1.004193    0.863320      0.100404  0.982581   
4  0.096626   0.460670    0.694160   -0.230144      1.076159  0.186487   

   SOLIDITY  CONVEX_AREA    EXTENT  ASPECT_RATIO  ROUNDNESS  COMPACTNESS  \
0  0.470574    -0.545399  0.581144     -0.045698   0.749949     0.731939   
1 -0.052512     0.728089  0.779211     -0.043494  -0.197979     0.478413   
2  0.092502    -1.372113  0.113331     -0.037087   0.366330    -0.052115   
3  0.051069     1.039762  0.411373     -0.039061  -0.498202     0.008919   
4 -0.787940     0.143179 -1.831492     -0.023671  -1.223743    -1.086566   

   SHAPEFACTOR_1  SHAPEFACTOR_2  SHAPEFACTOR_3  SHAPEFACTOR_4    MeanRR  \
0      -0.039050       

# Q-6


* Build a classifier with the training set.
## Initialize the classifier

In [32]:
classifier = RandomForestClassifier(random_state=42)

## Train the classifier with the training data

In [33]:
classifier.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

## Make predictions on the test set

In [34]:
y_pred = classifier.predict(X_test_scaled)

## Evaluate the classifier

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8611


## Print the classification report for detailed performance metrics

In [36]:

print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

       BERHI       0.91      0.71      0.80        14
      DEGLET       0.65      0.57      0.60        23
       DOKOL       0.86      0.90      0.88        41
       IRAQI       0.77      0.91      0.83        11
      ROTANA       1.00      0.96      0.98        26
      SAFAVI       0.94      1.00      0.97        45
       SOGAY       0.75      0.75      0.75        20

    accuracy                           0.86       180
   macro avg       0.84      0.83      0.83       180
weighted avg       0.86      0.86      0.86       180



# Q-7


* Predict class label of just one sample picked from the test set.
## Pick one sample from the test set (let's take the first sample)

In [37]:
sample = X_test_scaled.iloc[[0]]  # Use double square brackets to keep it as a DataFrame with column names

## Predict the class label for the selected sample

In [38]:
predicted_label = classifier.predict(sample)

## Print the predicted class label

In [39]:
print(f"Predicted class label: {predicted_label[0]}")

Predicted class label: IRAQI


## Also print the actual label for comparison

In [40]:

actual_label = y_test.iloc[0]
print(f"Actual class label: {actual_label}")

Actual class label: IRAQI


# Q-8


* Evaluate the model performance based on the test set.

## Make predictions on the test set

Explanation:

    accuracy_score: Computes the percentage of correct predictions.
    classification_report: Displays precision, recall, F1-score, and support for each class.
    confusion_matrix: Shows a matrix with true vs. predicted values for each class.

Example output:

    Accuracy: A single number indicating the overall performance.
    Classification Report: A breakdown of precision, recall, and F1-score for each class.
    Confusion Matrix: A matrix showing how well the model predicted each class.

In [42]:
y_pred = classifier.predict(X_test_scaled)

## 1. Accuracy

In [43]:

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8611


## 2. Classification report (precision, recall, f1-score)

In [44]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

       BERHI       0.91      0.71      0.80        14
      DEGLET       0.65      0.57      0.60        23
       DOKOL       0.86      0.90      0.88        41
       IRAQI       0.77      0.91      0.83        11
      ROTANA       1.00      0.96      0.98        26
      SAFAVI       0.94      1.00      0.97        45
       SOGAY       0.75      0.75      0.75        20

    accuracy                           0.86       180
   macro avg       0.84      0.83      0.83       180
weighted avg       0.86      0.86      0.86       180



## 3. Confusion matrix

In [45]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[10  0  0  3  0  0  1]
 [ 0 13  6  0  0  1  3]
 [ 0  4 37  0  0  0  0]
 [ 1  0  0 10  0  0  0]
 [ 0  0  0  0 25  0  1]
 [ 0  0  0  0  0 45  0]
 [ 0  3  0  0  0  2 15]]


# Q-9


* Can you do **better**? Here, **better** is a very subjective term :(

# 1. Hyperparameter Tuning

* Fine-tuning the hyperparameters of the model often leads to better performance. 
* For a Random Forest, you can optimize parameters such as the number of trees, the maximum depth of the trees, or the minimum samples per leaf. 

You can use GridSearchCV or RandomizedSearchCV for this:
# Define the parameter grid

In [53]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

## Modify the grid search to use a single job (no parallelism)

In [54]:
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=1)


## Fit the grid search to the data

In [55]:
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

## Get the best parameters

In [56]:
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}


## Use the best model to predict

In [57]:
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_scaled)

# Evaluate the model again

In [58]:

print(f"Accuracy after tuning: {accuracy_score(y_test, y_pred_best):.4f}")
print("Classification Report after tuning:")
print(classification_report(y_test, y_pred_best))

Accuracy after tuning: 0.8611
Classification Report after tuning:
              precision    recall  f1-score   support

       BERHI       0.92      0.79      0.85        14
      DEGLET       0.59      0.57      0.58        23
       DOKOL       0.86      0.88      0.87        41
       IRAQI       0.83      0.91      0.87        11
      ROTANA       1.00      0.96      0.98        26
      SAFAVI       0.96      1.00      0.98        45
       SOGAY       0.75      0.75      0.75        20

    accuracy                           0.86       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.86      0.86      0.86       180



# 2. Try Different Algorithms

Random Forest is a powerful model, but trying different classifiers may yield better results. 
    Support Vector Machines (SVM)
    Gradient Boosting (e.g., XGBoost, LightGBM)
    K-Nearest Neighbors (KNN)
    Logistic Regression

For example, you can try an SVM:

## Initialize the SVM classifier

In [60]:
svm_classifier = SVC(kernel='linear')

## Train the model

In [61]:
svm_classifier.fit(X_train_scaled, y_train)

SVC(kernel='linear')


## Predict and evaluate

In [62]:
y_pred_svm = svm_classifier.predict(X_test_scaled)
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.9222
SVM Classification Report:
              precision    recall  f1-score   support

       BERHI       1.00      0.93      0.96        14
      DEGLET       0.71      0.74      0.72        23
       DOKOL       0.95      0.90      0.93        41
       IRAQI       0.92      1.00      0.96        11
      ROTANA       1.00      1.00      1.00        26
      SAFAVI       1.00      1.00      1.00        45
       SOGAY       0.81      0.85      0.83        20

    accuracy                           0.92       180
   macro avg       0.91      0.92      0.91       180
weighted avg       0.92      0.92      0.92       180



In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cross_val_scores = cross_val_score(classifier, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validated accuracy: {cross_val_scores.mean():.4f}")


# 3. Feature Engineering

    Feature selection: You Experiment with selecting the most important features or removing irrelevant ones. This can reduce noise and improve performance.
    Create new features: Sometimes, combining or creating new features can give more meaningful inputs to the model.

# 4. Data Augmentation

    Create synthetic samples or use oversampling techniques (e.g., SMOTE) to balance the dataset if it's imbalanced.

# 5. Cross-Validation

    Instead of splitting the data into just training and test sets, use k-fold cross-validation to get a better estimate of model performance.
 ## Perform cross-validation


In [64]:
cross_val_scores = cross_val_score(classifier, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-validated accuracy: {cross_val_scores.mean():.4f}")


Cross-validated accuracy: 0.8900
